In [1]:
from collections import defaultdict
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import yfinance as yf
from finta import TA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from tabulate import tabulate
from ta import add_all_ta_features
import xgboost as xgb

In [18]:
WINDOW = 8  # number of rows to look ahead to see what the price did
FETCH_INTERVAL = "60m"  # fetch data by interval (including intraday if period < 60 days)
# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
# (optional, default is '1d')
INTERVAL = '2y'  # use "period" instead of start/end
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# (optional, default is '1mo')
symbol = 'AAPL'  # Symbol of the desired stock
ROWS_TO_PREDICT = 128
# one day 16 rows of data

In [19]:
data = pd.read_csv(
    'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\preprocess\\AAPL_16_21_04_2021 00_40_43_full.csv')

In [4]:
important_columns = ['open', 'high', 'low']

In [5]:

# def calculate_diffs(diff_number, col_name):
#     new_col_name = f'{col_name}_{diff_number}'
#     data[new_col_name] = data[col_name].diff(diff_number)
#

In [6]:
# for name in important_columns:
#     for i in range(1,11):
#         calculate_diffs(i,name)

In [20]:
# del (data['close'])
# del (data['open'])
# del (data['high'])
# del (data['volume'])
del (data['close_shift'])
data = data.dropna()
train_set = data.iloc[:-ROWS_TO_PREDICT]
train_set = train_set.iloc[:-WINDOW] # optional drop last n rows (avoid of data leak)
test_set =data.iloc[-ROWS_TO_PREDICT:]

In [21]:
data['class_column'].value_counts()

 1    1398
 0    1396
-1    1388
Name: class_column, dtype: int64

In [9]:
train_set

,Unnamed: 0,open,high,low,close,Adj Close,volume,close_pct,class_column,volume_adi,...,low_1,low_2,low_3,low_4,low_5,low_6,low_7,low_8,low_9,low_10
10,10,55.497501,56.161598,54.549999,55.264999,55.264999,7868884,-0.003988,1,-9.811428e+06,...,0.077499,1.397499,1.137501,-0.273750,-2.337501,-1.075001,-1.492501,-1.387501,-0.820001,-0.950001
11,11,55.247501,55.596424,54.255001,54.432499,54.432499,6766480,-0.015064,1,-1.478722e+07,...,-0.294998,-0.217499,1.102501,0.842503,-0.568748,-2.632499,-1.369999,-1.787499,-1.682499,-1.114999
12,12,54.435001,56.700001,54.365250,55.994999,55.994999,8603180,0.028705,1,-1.137967e+07,...,0.110249,-0.184750,-0.107250,1.212749,0.952751,-0.458500,-2.522250,-1.259750,-1.677250,-1.572250
13,13,56.092500,56.222500,55.269924,56.000000,56.000000,0,0.000089,1,-1.137967e+07,...,0.904674,1.014923,0.719925,0.797424,2.117424,1.857426,0.446174,-1.617576,-0.355076,-0.772576
14,14,55.912500,56.250000,55.800026,56.200000,56.200000,0,0.003571,1,-1.137967e+07,...,0.530102,1.434776,1.545025,1.250027,1.327526,2.647526,2.387528,0.976276,-1.087474,0.175026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4042,4042,121.849998,121.949997,119.794998,119.930000,119.930000,34515975,-0.016806,1,6.971709e+08,...,-0.245002,-0.205002,0.104998,0.094998,-0.355002,-0.205002,-1.255002,6.968138,6.454428,-1.025002
4043,4043,119.919998,120.489998,119.470001,120.429298,120.429298,18588318,0.004163,1,7.135468e+08,...,-0.324997,-0.569999,-0.529999,-0.219999,-0.229999,-0.679999,-0.529999,-1.579999,6.643141,6.129431
4044,4044,120.419998,120.500000,119.794998,119.861000,119.861000,11044540,-0.004719,1,7.045703e+08,...,0.324997,0.000000,-0.245002,-0.205002,0.104998,0.094998,-0.355002,-0.205002,-1.255002,6.968138
4045,4045,119.867500,120.410004,119.550003,120.327003,120.327003,13045822,0.003888,1,7.150979e+08,...,-0.244995,0.080002,-0.244995,-0.489997,-0.449997,-0.139997,-0.149997,-0.599997,-0.449997,-1.499997


In [22]:
y = data['class_column']
features = [x for x in data.columns if x not in ['class_column']]
x = data[features]
scaler = MinMaxScaler()
x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)
x_train= x.iloc[:-ROWS_TO_PREDICT]
y_train= y.iloc[:-ROWS_TO_PREDICT]
x_test =x.iloc[-ROWS_TO_PREDICT:]
y_test=y.iloc[-ROWS_TO_PREDICT:]

In [17]:
# for i in range(2,50):
#     model = xgb.XGBClassifier(nthread =-1,max_depth=i,n_estimators=1000,
#                           eta =0.2)
#     model.fit(x_train,y_train)
#     predicted_train = model.predict(x_train)
#     predicted_test = model.predict(x_test)
#     print("------------")
#     print(f'max_depth: {i}')
#     print(accuracy_score(y_train.values, predicted_train))
#     print(accuracy_score(y_test.values, predicted_test))
#     print("------------")

C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:30:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
------------
max_depth: 2
0.9985166872682324
0.40625
------------
[23:30:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
------------
max_depth: 3
1.0
0.4375
------------
[23:31:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_m

In [12]:
# predicted_test
#
#

In [13]:
# accuracy_score(y_train.values, predicted_train)
#
#

In [14]:
# accuracy_score(y_test.values, predicted_test)
#

In [15]:
# model.feature_importances_

In [23]:
model = xgb.XGBClassifier(nthread =-1,max_depth=14,n_estimators=1000,
                          eta =0.2)
model.fit(x_train,y_train)
predicted_train = model.predict(x_train)
predicted_test = model.predict(x_test)
print("------------")
print(f'max_depth: {14}')
print(accuracy_score(y_train.values, predicted_train))
print(accuracy_score(y_test.values, predicted_test))
print("------------")

C:\Users\exomat\anaconda3\envs\magisterka_analiza\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:24:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
------------
max_depth: 14
1.0
0.546875
------------
